<a href="https://colab.research.google.com/github/KorneliuszDyszczynski/ColorizingPhotosNeuralNetwork/blob/PlanA/KolorajzerV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('theblackmamba31/landscape-image-colorization')

In [ ]:
!unzip landscape-image-colorization.zip

In [14]:
!mv landscape\ Images/color data

In [16]:
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile

In [17]:
path = '//content/data/'
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [18]:
train = train_datagen.flow_from_directory(path, target_size=(256, 256),batch_size=700,class_mode=None)
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0])
      Y.append(lab[:,:,1:] / 128)
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

Found 7129 images belonging to 2 classes.
(700, 256, 256, 1)
(700, 256, 256, 2)


In [19]:
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=encoder_input, outputs=decoder_output)

In [21]:
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.fit(X,Y,validation_split=0.2, epochs=10, verbose=1 )

Epoch 1/10
18/18 [==============================] - 14s 704ms/step - loss: 0.0141 - accuracy: 0.5575 - val_loss: 0.0156 - val_accuracy: 0.6049
Epoch 2/10
18/18 [==============================] - 12s 691ms/step - loss: 0.0145 - accuracy: 0.5657 - val_loss: 0.0124 - val_accuracy: 0.5207
Epoch 3/10
18/18 [==============================] - 12s 693ms/step - loss: 0.0154 - accuracy: 0.6228 - val_loss: 0.0112 - val_accuracy: 0.6244
Epoch 4/10
18/18 [==============================] - 12s 691ms/step - loss: 0.0144 - accuracy: 0.6181 - val_loss: 0.0113 - val_accuracy: 0.6226
Epoch 5/10
18/18 [==============================] - 12s 692ms/step - loss: 0.0126 - accuracy: 0.6626 - val_loss: 0.0109 - val_accuracy: 0.6345
Epoch 6/10
18/18 [==============================] - 12s 693ms/step - loss: 0.0125 - accuracy: 0.6700 - val_loss: 0.0107 - val_accuracy: 0.6374
Epoch 7/10
18/18 [==============================] - 12s 691ms/step - loss: 0.0104 - accuracy: 0.6710 - val_loss: 0.0112 - val_accuracy: 0.6404

In [45]:
!rm -r //content/test/.ipynb_checkpoints

In [46]:
test_path = '//content/test/'
test = os.listdir(test_path)
for imgName in test:
  color_me = []
  img = img_to_array(load_img(test_path + imgName))
  img = resize(img ,(256,256))
  color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
print(color_me.shape)

(1, 256, 256, 1)


In [47]:
output = model.predict(color_me)
output = output * 128

In [48]:
# Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    imsave("result"+str(i)+".png", lab2rgb(result))